In [0]:
# update_all_dss_projects_on_github.py
# Tim H 2025
#
# This script iterates through all projects on a DSS instance and attempts to push all commits to GitHub
# for projects that are connected to GitHub.
# It is intended to be run as a scenario on a schedule
#
# https://developer.dataiku.com/latest/tutorials/devtools/using-api-with-git-project/index.html
# https://developer.dataiku.com/latest/api-reference/python/projects.html#dataikuapi.dss.project.DSSProjectGit.get_remote

import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
client = dataiku.api_client()
projects = client.list_projects()

In [0]:
successful = set()
not_connected = set()
errored = set()

projects_analysis = []

for iter_project_key in client.list_project_keys():
    try:
        proj = client.get_project(iter_project_key)
        project_git = proj.get_project_git()
        r = project_git.get_remote()
        status = project_git.get_status()
                
        has_github_repo = len(status.get('remotes',[])) > 0
        projects_analysis.append({"key":iter_project_key, "has_github_repo": has_github_repo, "status": str(status), "remote": r})

        if has_github_repo:
            res_push = project_git.push()
            res_pull = project_git.pull() # code studiodoes onot exist for PMM
            
            if (not res_push.get('success',False)) or (not res_pull.get('success',False)):
#             if not res_push.get('success',False):
                print(f"[ERROR] pushing or pulling {iter_project_key}")
                errored.add(iter_project_key)
                continue
            successful.add(iter_project_key)
        else:
            # print(f"{iter_project_key} is not connected to GitHub")
            not_connected.add(iter_project_key)
    except Exception as e:
        print(f"[EXCEPTION] {iter_project_key}: {e}")
        errored.add(iter_project_key)
        continue
        
pd.DataFrame(projects_analysis).sort_values(by=["has_github_repo", "remote"])

In [0]:
successful

In [0]:
not_connected

In [0]:
errored